In [109]:
import pandas as pd
import numpy as np
import datetime

In [111]:
columns = ['month', 'day', 'time', 'i_cost', 'i_1p', 'i_climate', 'abs_comp', 'abs_opt', 'rel_comp', 'rel_opt']
gerben = pd.read_csv('gerben_output/aa_high_potential_list_sorted_rel_climate.txt', header=None, names=columns)
atr_array =pd.read_csv('atr_output/ATRsum-July-1-week.csv')

date_list = []

#converts the dates into a normal format
for i in range(len(gerben)):
    date_list.append(datetime.date(2018, gerben['month'].values[i], 1 + gerben['day'].values[i]))
#converts from datetime.date to string
date_strings = [date.strftime("%Y-%m-%d") for date in date_list]
#drops the useless month/day
gerben_no_date = gerben.drop(axis=1, columns=['month', 'day'])
#inserts better string date
gerben_no_date.insert(loc=0, column='Date', value=date_strings)
#yeets out the crap we won't use
gerben_new = gerben_no_date.drop(axis=1, columns=['abs_comp', 'abs_opt', 'rel_comp'])
gerben_midnight = gerben_new[gerben_new['time'] != 1]
#funny dict goes brrrrrr
gerben_day = {}
atr_day = {}
#fill up the dicts
for i in range(1,8):
    gerben_day['gerben_day_{0}'.format(i)] = gerben_midnight[gerben_midnight['Date'] == '2018-07-0' + str(i)]
    atr_day['atr_day_{0}'.format(i)] = atr_array[atr_array['Date'] == '2018-07-0' + str(i)]
#sorts by values descending so it matches the gerben output in terms of correlation, i.e. the lower the rel_opt the higher ATRsum
for i in range(1,8):
    atr_day_temp = atr_day.get('atr_day_1')
    atr_day_temp = atr_day_temp.sort_values(by='ATRsum', ascending=False)
    atr_day['atr_day_{0}'.format(i)] = atr_day_temp

#calculate the amount of flights that lie in the same halve
total_values = pd.DataFrame(columns=['Date', 'Correlation_Factor'])

for i in range(1,8):
    midpoint = len(atr_day.get('atr_day_' + str(i))) // 2

    index_atr_1st_half = atr_day.get('atr_day_' + str(i))['Flight_index'][:midpoint]
    index_atr_2nd_half = atr_day.get('atr_day_' + str(i))['Flight_index'][midpoint:]

    index_gerben_1st_half = gerben_day.get('gerben_day_' + str(i))['i_climate'][:midpoint]
    index_gerben_2nd_half = gerben_day.get('gerben_day_' + str(i))['i_climate'][midpoint:]

    intersection_1st = set(index_atr_1st_half).intersection(set(index_gerben_1st_half))
    intersection_2nd = set(index_atr_2nd_half).intersection(set(index_gerben_2nd_half))
    total = float(len(intersection_1st) + len(intersection_2nd))/100
    total_values = total_values.append({'Date': '2018-07-0' + str(i), 'Correlation_Factor': total}, ignore_index=True)

#calculate the average over the entire week
average = total_values['Correlation_Factor'].mean()

print(total_values)
print('Average =', average)


         Date  Correlation_Factor
0  2018-07-01                0.48
1  2018-07-02                0.64
2  2018-07-03                0.60
3  2018-07-04                0.58
4  2018-07-05                0.56
5  2018-07-06                0.56
6  2018-07-07                0.50
Average = 0.56


C:\Users\macie\AppData\Local\Temp\ipykernel_2632\1989905475.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_values = total_values.append({'Date': '2018-07-0' + str(i), 'Correlation_Factor': total}, ignore_index=True)
C:\Users\macie\AppData\Local\Temp\ipykernel_2632\1989905475.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_values = total_values.append({'Date': '2018-07-0' + str(i), 'Correlation_Factor': total}, ignore_index=True)
C:\Users\macie\AppData\Local\Temp\ipykernel_2632\1989905475.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_values = total_values.append({'Date': '2018-07-0' + str(i), 'Correlation_Factor': total}, ignore_index=True)
C:\Users\macie\AppData\Local\Temp\ipykernel_2632\